In [1]:
weather_pth = r'C:\Users\anat.shkolyar\PycharmProjects\pandas_workshop\data\weather.db'

In [2]:
aapl_pth = r'C:\Users\anat.shkolyar\PycharmProjects\pandas_workshop\data\AAPL.csv.bz2'

In [3]:
%matplotlib inline
import pandas as pd

In [4]:
df = pd.read_csv(aapl_pth, parse_dates=['Date'], index_col='Date')
df.head()  # Apple stock price at different points in the day (columns)

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-06-17,96.620003,96.650002,95.300003,95.330002,60595000,95.330002
2016-06-16,96.449997,97.750000,96.070000,97.550003,31236300,97.550003
2016-06-15,97.820000,98.410004,97.029999,97.139999,29254300,97.139999
2016-06-14,97.320000,98.480003,96.750000,97.459999,31870300,97.459999
2016-06-13,98.690002,99.120003,97.099998,97.339996,37612900,97.339996


In [5]:
# How many trading days in 2010
len(df.loc['2010'])

252

In [7]:
# range
len(df.loc['2010-12-31':'2008-01-01'])  # order is reversed (2010:2008) because index is in descending order

757

In [8]:
# get weather data from different file - from sqlite file (database in single file)
# sqlite saves data as string and data is parsed when fetched
import sqlite3

In [9]:
# in python we have spec for DB api. this allows pandas to work with connection objects which is same for any DB type.
conn = sqlite3.connect(weather_pth)  # connect to ':memory' to get in memory database

In [10]:
# you can also infer type of columns in connection object - see sqlite3 docs
wdf = pd.read_sql('SELECT * FROM weather', conn, parse_dates=['DATE'], index_col='DATE')  # use SQL queries in database
wdf.head()

# note that TMAX does not make sense. 178C \ 178F cannot be measures in central park. in weather.txt we have schema that notes that TMAX is in tenths of Celsius 178 = 17.8C 

,index,STATION,PRCP,SNWD,SNOW,TMAX,TMIN,AWND,WDF2,WDF5,WSF2,WSF5,PGTM,FMTM
DATE,,,,,,,,,,,,,,
2000-01-01,0,GHCND:USW00094728,0,-9999,0,100,11,26,250,230,72,94,1337,1337
2000-01-02,1,GHCND:USW00094728,0,-9999,0,156,61,21,260,260,72,112,2313,2314
2000-01-03,2,GHCND:USW00094728,0,-9999,0,178,106,30,260,250,67,94,320,321
2000-01-04,3,GHCND:USW00094728,178,-9999,0,156,78,35,320,350,67,107,1819,1840
2000-01-05,4,GHCND:USW00094728,0,-9999,0,83,-17,51,330,340,107,143,843,844


In [12]:
fdf = df.join(wdf)  # add columns from wdf to columns in df, if not provided, we're left joininig by index
# left \ right joining - if row does not exist in OTHER (right\left), do not add
# outer join - if row does not exist, add with NaN
# inner join - only join if row exists in both
# see pd documentation of join \ merge for details
fdf.head()

,Open,High,Low,Close,Volume,Adj Close,index,STATION,PRCP,SNWD,SNOW,TMAX,TMIN,AWND,WDF2,WDF5,WSF2,WSF5,PGTM,FMTM
Date,,,,,,,,,,,,,,,,,,,,
2016-06-17,96.620003,96.650002,95.300003,95.330002,60595000,95.330002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-16,96.449997,97.750000,96.070000,97.550003,31236300,97.550003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-15,97.820000,98.410004,97.029999,97.139999,29254300,97.139999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-14,97.320000,98.480003,96.750000,97.459999,31870300,97.459999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-13,98.690002,99.120003,97.099998,97.339996,37612900,97.339996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
len(fdf[pd.isnull(fdf['SNOW'])])  # many nulls, as sampling rate of weather and stocks are different

5365

In [14]:
def numna(name):
    print('#NaN after {}: {}'.format(name, len(fdf[pd.isnull(fdf['SNOW'])])))

numna('join')

#NaN after join: 5365


In [15]:
fdf.interpolate(inplace=True)  # inplace True changes fdf and not only shows view
numna('interpolate')  # edges cannot be filled by interpolation

#NaN after interpolate: 424


In [16]:
fdf.fillna(0, inplace=True)  # fill NaN with 0, only for numeric columns
numna('fill')

#NaN after fill: 0


In [18]:
fdf[['SNOW', 'Close']].corr()  # no correlation between snow and apple stock price

,SNOW,Close
SNOW,1.000000,0.063585
Close,0.063585,1.000000


In [ ]:
# python for data analysis 2nd edition - book, good for pandas
# python data science handbook (VanderPlas) - free online!